# Quick Start Guide - Azure AI Foundry

This notebook provides a hands-on introduction to Azure AI Foundry. You'll learn how to:
1. Initialize the AI Project client
2. List available models
3. Create a simple completion request
4. Handle basic error scenarios

## Prerequisites
- Completed environment setup from previous notebook
- Azure credentials configured

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials
credential = DefaultAzureCredential()

## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Copy your `.env.local` file to `.env`
> 2. Update the project connection string in your `.env` file
> 3. Have a Hub and Project already provisioned in Azure AI Foundry

You can find your project connection string in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



In [ ]:
from dotenv import load_dotenv

# Create AI Project client
# Load environment variables from .env file
load_dotenv()

try:
    client = AIProjectClient.from_connection_string(
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

## Create a Simple Completion
Let's try a basic completion request:

In [ ]:
from azure.ai.inference.models import UserMessage

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

try:
    chat_client = client.inference.get_chat_completions_client()
    response = chat_client.complete(
        model=model_deployment_name, 
        messages=[UserMessage(content="How to be healthy in one sentence?")]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {str(e)}")

## Create a simple Agent

Using AI Agent Service, we can create a simple agent to answer health related questions.

In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

try:
    # Create an agent with code interpreter
    code_interpreter = CodeInterpreterTool()
    agent = client.agents.create_agent(
        model=model_deployment_name,
        name="bmi-calculator",
        instructions="You are a health analyst who calculates BMI using US metrics (pounds, feet/inches). Use average US male measurements: 5'9\" (69 inches) and 198 pounds. Create a visualization showing where this BMI falls on the scale.",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    
    thread = client.agents.create_thread()
    
    # Request BMI analysis
    message = client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Calculate BMI for an average US male (5'9\", 198 lbs). Create a visualization showing where this BMI falls on the standard BMI scale from 15 to 35. Include the standard BMI categories (Underweight, Normal, Overweight, Obese) in the visualization."
    )
    
    # Process the request
    run = client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    
    # Get and save any visualizations
    messages = client.agents.list_messages(thread_id=thread.id)
    for image_content in messages.image_contents:
        file_name = f"bmi_analysis_{image_content.image_file.file_id}.png"
        client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Analysis saved as: {file_name}")
    
    # Print the analysis
    if last_msg := messages.get_last_text_message_by_sender("assistant"):
        print(f"Analysis: {last_msg.text.value}")
    
    # Cleanup
    client.agents.delete_agent(agent.id)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

## Error Handling Example
Let's see how to handle a common error scenario:

In [ ]:
# Test error handling
try:
    # Intentionally use invalid deployment name
    response = client.models.generate(
        deployment_name="non-existent-model",
        prompt="This should fail gracefully",
        max_tokens=100
    )
except Exception as e:
    print("Expected error caught successfully:")
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)}")

## Next Steps
- Explore more advanced model configurations
- Try different prompt types
- Implement retry logic for robustness
- Add logging for production use